for the first exploration, i will try to do vectorizer with toy dataset

In [ ]:
import pandas as pd 
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# this is a very toy example, do not try this at home unless you want to understand the usage differences 
docs=["the house had a tiny little mouse", 
"the cat saw the mouse", 
"the mouse ran away from the house", 
"the cat finally ate the mouse", 
"the end of the mouse story"
]

In [ ]:
docs

In [ ]:
#instantiate CountVectorizer() 
cv=CountVectorizer() 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(docs)

In [ ]:
word_count_vector.shape

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

In [ ]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1, 3))
X2 = vectorizer2.fit_transform(docs)
vectorizer2.get_feature_names_out()

In [ ]:
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(X2)

In [ ]:
# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=vectorizer2.get_feature_names(),columns=["idf_weights"]) 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

I grasped the theory a bit about feature extraction in text, next is cleaning i guess? The cleaning consist of : 
- case folding
- data cleaning (removes rt, username, url, punctuation, emoticon)
- text normalization (changes non formal words into the formal one)
- stemming
- stop words removal

# Data upload

In [ ]:
data = './repo_dataset/id-multi-label-hate-speech-and-abusive-language-detection/re_dataset.csv'

In [ ]:
df = pd.read_csv(data, encoding = "ISO-8859-1")

In [ ]:
df['Tweet']

In [ ]:
import re
import emoji

## Case Folding

In [ ]:
df['Tweet'] = df['Tweet'].str.lower()

In [ ]:
df['Tweet'][1]

data cleaning (removes rt, username, url, punctuation, emoticon)

In [ ]:
# data cleaning
dummy = 'rt user url betul sekali itu 12 !!!'


dummy = re.sub("rt","",dummy)
dummy = re.sub("user","",dummy)
dummy = re.sub("url","",dummy)


mod_string = ""
for elem in dummy:
    if elem.isalnum() or elem == ' ':
        mod_string += elem
dummy = mod_string


dummy = " ".join(dummy.split())
print(dummy)

In [ ]:
def clean(tweet):

    tweet = re.sub("rt","", str(tweet))
    tweet = re.sub("user","",str(tweet))
    tweet = re.sub("url","",str(tweet))
    tweet = re.sub(r'[^\w\s]', '', str(tweet)) # punctuation
    tweet = re.sub(r"\d+", "", str(tweet)) # number
    


    mod_string = ""
    for elem in tweet:
        if elem.isalnum() or elem == ' ':
            mod_string += elem
    tweet = mod_string
    
    res = emoji.emoji_list(tweet)
    if len(res) > 0 :
        tweet = emoji.replace_emoji(tweet, replace='')

    tweet = " ".join(tweet.split())
    return tweet

In [ ]:
df['Tweet'] = df['Tweet'].map(lambda x: clean(x))

In [ ]:
df

text normalization (changes non formal words into the formal one)

In [ ]:
reference = './repo_dataset/id-multi-label-hate-speech-and-abusive-language-detection/new_kamusalay.csv'

In [ ]:
df_ref = pd.read_csv(reference, encoding = "ISO-8859-1", header=None)

In [ ]:
df_ref

In [ ]:
dummy = '3x ke kamar menaker mandi'
# for item in df_ref[0]:
#     print(item)

tempDum = dummy.split()

# for temp in tempdum:
#     for item in df_ref[0]:
#         if temp == item:
#             print(temp)
#             print(item)
            
for tempInd, tempVal in enumerate(tempDum):
    for ind, item in enumerate(df_ref[0]):
        if tempVal == item:
            print(item)
            print(df_ref[1][ind])
            tempDum[tempInd] = df_ref[1][ind]
            
dummy = " ".join(tempDum)

print(dummy)

In [ ]:
def text_norm(text):
    tempText = text.split()
    
    for tempInd, tempVal in enumerate(tempText):
        for ind, item in enumerate(df_ref[0]):
            if tempVal == item:
                tempText[tempInd] = df_ref[1][ind]
    
    text = " ".join(tempText)
    return text

In [ ]:
df['Tweet'] = df['Tweet'].map(lambda x: text_norm(x))

In [ ]:
df

In [ ]:
df.to_csv('./repo_dataset/id-multi-label-hate-speech-and-abusive-language-detection/out_clean1.csv')


Continue where i left off

In [ ]:
checkpoint = './repo_dataset/id-multi-label-hate-speech-and-abusive-language-detection/out_clean1.csv'

In [ ]:
df_check = pd.read_csv(checkpoint, encoding = "ISO-8859-1")

In [ ]:
df_check = df_check.drop("Unnamed: 0",axis=1)

In [ ]:
df_check

kinda forget to remove numerical and punctuation

stemming

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()
# stemming process
sentence = 'Perekonomian Indonesia sedang dalam pertumbuhan yang membanggakan'
output   = stemmer.stem(sentence)
print(output)

In [ ]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# from nltk.tokenize import word_tokenize
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()
kalimat = "Andi kerap melakukan transaksi rutin secara daring atau online. Menurut Andi belanja online lebih praktis & murah."
# kalimat = kalimat.translate(str.maketrans('','',string.punctuation)).lower()
stop = stopword.remove(kalimat)
# tokens = nltk.tokenize.word_tokenize(stop)
print(stop)

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()
factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def stemming(text):

    output = stemmer.stem(text)
    return output

def stopWordsRemoval(text):

    output = stopword.remove(text)
    
    return output

In [ ]:
df_check['Tweet'] = df_check['Tweet'].map(lambda x: stemming(x))
df_check['Tweet'] = df_check['Tweet'].map(lambda x: stopWordsRemoval(x))

In [ ]:
df_check

In [ ]:
df_check.to_csv('./repo_dataset/id-multi-label-hate-speech-and-abusive-language-detection/out_clean2.csv')